# Notes: Circuit Manipulation

### by ReDay Zarra

## Setup

In [1]:
import sys
print(sys.version)

3.9.17 (main, Jun  6 2023, 20:11:21) 
[GCC 11.3.0]


In [2]:
import importlib, pkg_resources
importlib.reload(pkg_resources)

<module 'pkg_resources' from '/usr/lib/python3/dist-packages/pkg_resources/__init__.py'>

In [3]:
import tensorflow as tf
import tensorflow_quantum as tfq

import cirq
import sympy
import numpy as np

2023-07-02 12:56:37.121980: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-02 12:56:37.122029: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
2023-07-02 12:56:41.693866: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-07-02 12:56:41.693932: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-07-02 12:56:41.693976: I tensorflow/stream_executor/cuda/cuda_diagnostics

In [4]:
# Visualization tools: matplotlib and Cirq's svg circuit
%matplotlib inline
import matplotlib.pyplot as plt
from cirq.contrib.svg import SVGCircuit

In [5]:
print("TensorFlow version: ", tf.__version__)
print("TensorFlow Quantum version: ", tfq.__version__)
print("Cirq version: ", cirq.__version__)

TensorFlow version:  2.7.0
TensorFlow Quantum version:  0.7.2
Cirq version:  1.1.0


## Circuit Manipulation

### Patterns for Arguments

Generator: a Python function that uses the yield keyword to return an iterable sequence of items.

In Cirq, `my_layer()` is a generator function that yields a sequence of operations and collections of operations. Each `yield` statement in the function produces a new item in the sequence. These items can be individual operations, lists of operations, or nested lists of operations.

In [8]:
# Define qubits
q0, q1, q2 = [cirq.GridQubit(i, 0) for i in range(3)]

# Produces a series of quantum operations and collections of operations.
def my_layer():
    yield cirq.CZ(q0, q1)
    yield [cirq.H(q) for q in (q0, q1, q2)]
    yield [cirq.CZ(q1, q2)]
    yield [cirq.H(q0), [cirq.CZ(q1, q2)]]

# Create new circuit and add all operations yielded by the generator
circuit = cirq.Circuit()
circuit.append(my_layer())

for x in my_layer():
    print(x)

CZ(q(0, 0), q(1, 0))
[cirq.H(cirq.GridQubit(0, 0)), cirq.H(cirq.GridQubit(1, 0)), cirq.H(cirq.GridQubit(2, 0))]
[cirq.CZ(cirq.GridQubit(1, 0), cirq.GridQubit(2, 0))]
[cirq.H(cirq.GridQubit(0, 0)), [cirq.CZ(cirq.GridQubit(1, 0), cirq.GridQubit(2, 0))]]


In [9]:
print(circuit)

(0, 0): ───@───H───H───────
           │
(1, 0): ───@───H───@───@───
                   │   │
(2, 0): ───H───────@───@───


The `Circuit.append` (or `Circuit.insert`) is able to accept an iterator and turn it into a simple, one-dimensional list of operations. This process is called "**flattening**", because it takes a potentially complex, multi-layered structure and **turns it into a simple list**.

`cirq.OP_TREE`: it's a kind of contract or promise about the structure of the data. If you have an iterable structure (like a list, or a generator function) that **can be flattened into a list of operations** (or Moment objects, which are themselves just collections of operations), **then that structure is considered an** `OP_TREE`.

**Sub-circuits**: is just a **part of a larger quantum circuit**, and a generator function is a convenient way to generate these sub-circuits on the fly. Because these generator functions can accept arguments, you can easily adjust the size of the sub-circuits or the parameters of the operations within them, which can make your code much more flexible and powerful.

In [11]:
circuit = cirq.Circuit(cirq.H(q0), cirq.H(q1))
print(circuit)

(0, 0): ───H───

(1, 0): ───H───
